In [9]:
import pandas as pd

datafile = "Finance_Trends.csv"
inputTable = pd.read_csv(datafile)
inputTable


,gender,age,Investment_Avenues,Mutual_Funds,Equity_Market,Debentures,Government_Bonds,Fixed_Deposits,PPF,Gold,...,Duration,Invest_Monitor,Expect,Avenue,What are your savings objectives?,Reason_Equity,Reason_Mutual,Reason_Bonds,Reason_FD,Source
0,Male,29,Yes,2,4,7,5,3,1,6,...,Less than 1 year,Weekly,20%-30%,Public Provident Fund,Health Care,Dividend,Fund Diversification,Assured Returns,Fixed Returns,Newspapers and Magazines
1,Male,28,Yes,2,3,6,5,1,4,7,...,Less than 1 year,Weekly,30%-40%,Public Provident Fund,Retirement Plan,Capital Appreciation,Better Returns,Assured Returns,Risk Free,Television
2,Female,28,Yes,2,3,7,5,4,1,6,...,3-5 years,Daily,10%-20%,Mutual Fund,Health Care,Dividend,Better Returns,Assured Returns,Risk Free,Financial Consultants
3,Male,19,Yes,2,3,7,4,6,1,5,...,More than 5 years,Monthly,10%-20%,Equity,Retirement Plan,Capital Appreciation,Better Returns,Assured Returns,Risk Free,Newspapers and Magazines
4,Male,32,Yes,2,4,7,5,3,1,6,...,More than 5 years,Weekly,10%-20%,Public Provident Fund,Retirement Plan,Capital Appreciation,Better Returns,Assured Returns,Fixed Returns,Financial Consultants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,Male,28,Yes,2,4,7,5,3,1,6,...,3-5 years,Daily,30%-40%,Fixed Deposits,Health Care,Dividend,Fund Diversification,Assured Returns,Fixed Returns,Newspapers and Magazines
11996,Male,29,Yes,2,4,7,5,3,1,6,...,More than 5 years,Daily,20%-30%,Fixed Deposits,Retirement Plan,Capital Appreciation,Better Returns,Assured Returns,Fixed Returns,Financial Consultants
11997,Female,34,Yes,5,4,3,2,7,1,6,...,Less than 1 year,Daily,20%-30%,Equity,Retirement Plan,Capital Appreciation,Tax Benefits,Safe Investment,Fixed Returns,Newspapers and Magazines
11998,Female,24,No,1,2,3,4,5,6,7,...,3-5 years,Daily,10%-20%,Public Provident Fund,Education,Dividend,Better Returns,Safe Investment,Risk Free,Internet


In [10]:
#Randomly Remove Values and create outliers

import numpy as np

numericColumns = inputTable.select_dtypes(include='number').columns.tolist()

#Randomly remove 20% of the data
for column in inputTable:
    randomCells = np.random.rand(len(inputTable)) < 0.2
    inputTable.loc[randomCells, column] = np.nan

#Randomly create outliers of 10% of the data
for column in inputTable.select_dtypes(include='number'):
    randomCells = np.random.rand(len(inputTable)) < 0.1
    inputTable.loc[randomCells, column] = inputTable.loc[randomCells, column] * 100

inputTable.head()

,gender,age,Investment_Avenues,Mutual_Funds,Equity_Market,Debentures,Government_Bonds,Fixed_Deposits,PPF,Gold,...,Duration,Invest_Monitor,Expect,Avenue,What are your savings objectives?,Reason_Equity,Reason_Mutual,Reason_Bonds,Reason_FD,Source
0,NaN,29.0,NaN,2.0,4.0,7.0,5.0,3.0,1.0,6.0,...,Less than 1 year,Weekly,20%-30%,Public Provident Fund,Health Care,Dividend,NaN,Assured Returns,Fixed Returns,NaN
1,Male,2800.0,Yes,2.0,3.0,6.0,5.0,1.0,4.0,7.0,...,Less than 1 year,NaN,30%-40%,Public Provident Fund,Retirement Plan,Capital Appreciation,NaN,Assured Returns,Risk Free,Television
2,Female,NaN,Yes,2.0,3.0,7.0,500.0,NaN,1.0,NaN,...,3-5 years,NaN,NaN,Mutual Fund,NaN,Dividend,Better Returns,Assured Returns,NaN,Financial Consultants
3,Male,19.0,Yes,2.0,3.0,NaN,NaN,6.0,1.0,5.0,...,More than 5 years,Monthly,10%-20%,Equity,Retirement Plan,Capital Appreciation,NaN,Assured Returns,Risk Free,NaN
4,Male,32.0,Yes,2.0,NaN,7.0,5.0,300.0,1.0,NaN,...,More than 5 years,Weekly,10%-20%,Public Provident Fund,Retirement Plan,NaN,Better Returns,NaN,Fixed Returns,Financial Consultants


In [11]:
#Transformation Table

inputTable.columns = [column.lower() for column in inputTable.columns]
numericColumns = []

#Find numeric columns by checking top ten values for numeric data
for col in inputTable:
    topTenValues = inputTable[col].dropna().head(10)
    numeric_count = pd.to_numeric(topTenValues, errors='coerce').notna().sum()

    if numeric_count > 0:
        numericColumns.append(col)

#Get rid of exceptional values
for column in numericColumns:
    inputTable[column] = pd.to_numeric(inputTable[column], errors="coerce")

    #Find the lower quantile of the column values
    lowerQuantile = inputTable[column].quantile(0.25)
    
    #Find the upper quantile of the column values
    upperQuantile = inputTable[column].quantile(0.75)
    
    #Find the inter quantile range of the column values
    interQuantileRange = upperQuantile - lowerQuantile

    lowerBound = lowerQuantile - 1.5 * interQuantileRange
    upperBound = upperQuantile + 1.5 * interQuantileRange

    #Seperate outliers using the bounds
    outliers = (inputTable[column] < lowerBound) | (inputTable[column] > upperBound)
    
    #Divide outliers by 100
    inputTable.loc[outliers, column] =  inputTable.loc[outliers, column] / 100

inputTable = inputTable.fillna('NA')

inputTable

,gender,age,investment_avenues,mutual_funds,equity_market,debentures,government_bonds,fixed_deposits,ppf,gold,...,duration,invest_monitor,expect,avenue,what are your savings objectives?,reason_equity,reason_mutual,reason_bonds,reason_fd,source
0,NA,29.0,NA,2.0,4.0,7.0,5.0,3.0,1.0,6.0,...,Less than 1 year,Weekly,20%-30%,Public Provident Fund,Health Care,Dividend,NA,Assured Returns,Fixed Returns,NA
1,Male,28.0,Yes,2.0,3.0,6.0,5.0,1.0,4.0,7.0,...,Less than 1 year,NA,30%-40%,Public Provident Fund,Retirement Plan,Capital Appreciation,NA,Assured Returns,Risk Free,Television
2,Female,NA,Yes,2.0,3.0,7.0,5.0,NA,1.0,NA,...,3-5 years,NA,NA,Mutual Fund,NA,Dividend,Better Returns,Assured Returns,NA,Financial Consultants
3,Male,19.0,Yes,2.0,3.0,NA,NA,6.0,1.0,5.0,...,More than 5 years,Monthly,10%-20%,Equity,Retirement Plan,Capital Appreciation,NA,Assured Returns,Risk Free,NA
4,Male,32.0,Yes,2.0,NA,7.0,5.0,3.0,1.0,NA,...,More than 5 years,Weekly,10%-20%,Public Provident Fund,Retirement Plan,NA,Better Returns,NA,Fixed Returns,Financial Consultants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,Male,NA,Yes,2.0,NA,7.0,5.0,3.0,1.0,6.0,...,NA,Daily,30%-40%,Fixed Deposits,Health Care,Dividend,NA,Assured Returns,Fixed Returns,Newspapers and Magazines
11996,Male,29.0,Yes,2.0,4.0,7.0,NA,3.0,1.0,6.0,...,More than 5 years,NA,20%-30%,Fixed Deposits,Retirement Plan,Capital Appreciation,Better Returns,NA,Fixed Returns,NA
11997,Female,NA,Yes,0.05,4.0,3.0,2.0,7.0,1.0,6.0,...,Less than 1 year,Daily,NA,Equity,NA,Capital Appreciation,Tax Benefits,Safe Investment,NA,Newspapers and Magazines
11998,Female,24.0,No,1.0,2.0,NA,NA,5.0,6.0,NA,...,NA,NA,10%-20%,Public Provident Fund,NA,Dividend,Better Returns,Safe Investment,Risk Free,Internet


In [ ]:
for columns not in numericColumns:
    print(input